# Testing CuDF's GPU acceleration w/ Pandas dataframes

Dataset used: UK property price data from Kaggle:

https://www.kaggle.com/datasets/willianoliveiragibin/uk-property-price-data-1995-2023-04/data

In [3]:
#import cudf

In [27]:
import pandas as pd
import numpy as np
import random

In [4]:
#%load_ext cudf.pandas

In [35]:
# Inspired by Rob Mulla - https://www.youtube.com/watch?v=u4_c2LDi4b8&list=PL7RwtdVQXQ8qxBH6ugYn50D0M5u--2Xx4&index=15
# Good result - a good result for a team in their category (e.g. points for backmarker, podium for midfield, win for leading)
# Assuming 21 races a season.
def rand_color(paint, base, base_col, size):
    '''
        Returns a string joining randomly picked elements from the three lists passed as args.
    '''
    cols_list = []
    for iteration in range(size):
        color = ''
        color = color + random.choice(paint)
        color = color + ' ' + random.choice(base)
        color = color + ' ' + random.choice(base_col)
        cols_list.append(color)
    print(cols_list[:10])
    return cols_list
def make_data(size):
    paint_type = ['Gloss', 'Matte', 'Iridescent', 'Semi-gloss', 'Satin']
    color_type = ['Light', 'Dark', 'Deep', 'Pale', 'Chromatic', 'Racing', 'Radiant']
    base_colors = ['Red', 'Green', 'Blue', 'White', 'Black', 'Yellow']
    df = pd.DataFrame()
    df['year'] = np.random.randint(1990, 2023, size)
    df['team'] = np.random.choice(['leading', 'midfield', 'backmarker'], size)
    df['best_finish'] = np.random.randint(1, 20, size)
    df['points_avg'] = np.random.randint(1, 26, size)
    df['total_points'] = np.multiply(df['points_avg'].to_numpy(), 21)
    df['color_type'] = rand_color(paint_type, color_type, base_colors, size)
    df['car_color'] = np.random.choice(paint_type + color_type + base_colors, size)
    df['good_res'] = np.random.choice(['yes', 'no'], size)
    df['prob_good_res'] = np.random.uniform(0, 1, size)
    return df

In [36]:
df2 = make_data(10 ** 4)

['Matte Racing Green', 'Satin Racing White', 'Semi-gloss Racing White', 'Matte Deep Green', 'Matte Dark Yellow', 'Semi-gloss Racing Yellow', 'Semi-gloss Radiant Green', 'Satin Deep Blue', 'Semi-gloss Racing Blue', 'Matte Racing White']


In [37]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           10000 non-null  int64  
 1   team           10000 non-null  object 
 2   best_finish    10000 non-null  int64  
 3   points_avg     10000 non-null  int64  
 4   total_points   10000 non-null  int64  
 5   color_type     10000 non-null  object 
 6   car_color      10000 non-null  object 
 7   good_res       10000 non-null  object 
 8   prob_good_res  10000 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 703.2+ KB


In [38]:
df2.to_csv('data.csv')
df = pd.read_csv('data.csv')

First, we'll import the data and look at it. 

Note the datatype for price. We're explicitly setting this, due to the original dataset being parsed as a string. 

In [17]:
len(df)

10000000

A simple operation to find unique values. In this case, we're going to grab for unique Towns/Cities.

In [39]:
# get unique values for a given column:
which_unique_col = 'color_type'
uniques = df[which_unique_col].unique()
print(len(uniques))

210


In [40]:
df['color_type']

0            Matte Racing Green
1            Satin Racing White
2       Semi-gloss Racing White
3              Matte Deep Green
4             Matte Dark Yellow
                 ...           
9995             Matte Dark Red
9996           Matte Deep White
9997          Gloss Deep Yellow
9998          Satin Pale Yellow
9999          Gloss Deep Yellow
Name: color_type, Length: 10000, dtype: object

In [21]:
uniques[:10]

array(['Semi-glossChromaticWhite'], dtype=object)

Now let's do a slightly more complicated operation. We're going to find the average price for each town/city.

In [42]:
df['best_finish'].unique()

array([14,  9,  6,  5, 16,  3, 18,  8, 11, 15,  4, 19, 13,  2,  7,  1, 10,
       12, 17])

In [42]:
%%timeit
years = sorted(df['year'].unique())

159 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [44]:
years = sorted(df['year'].unique())
print(years)
uniques_points = {}

[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [69]:
%%timeit
for unique in uniques:
    uniques_points[unique] = df[df[which_unique_col] == unique]['total_points'].mean()

228 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
for unique in uniques:
    uniques_points[unique] = df[df[which_unique_col] == unique]['total_points'].mean()
len(uniques_points.items())

210

In [65]:
#uniques_points.items()

Next, we'll grab data just for 2022:

In [47]:
# Use string or number in vectorized version
# df_2022 = df[df['year'].str.startswith('2022')]
df_green = df[df[which_unique_col] == 'Matte Racing Green']

In [49]:
print(len(df_green))
df_green.tail()

46


,Unnamed: 0,year,team,best_finish,points_avg,total_points,color_type,car_color,good_res,prob_good_res
8184,8184,2020,midfield,19,14,294,Matte Racing Green,Semi-gloss,yes,0.422315
8732,8732,1996,backmarker,4,11,231,Matte Racing Green,Pale,no,0.979515
9526,9526,2020,leading,1,6,126,Matte Racing Green,Green,yes,0.219553
9646,9646,2022,leading,9,12,252,Matte Racing Green,Satin,no,0.488504
9709,9709,1997,midfield,2,14,294,Matte Racing Green,Matte,yes,0.540164


Grab the lower 20% and upper 20% averages for properties in 2022.

In [67]:
recent_lower_upper = {}

In [68]:
%%timeit
for unique in uniques:

    points = df[df[which_unique_col] == unique]['total_points']
    sorted_points = points.sort_values()

    # bottom 20% of the prices
    bottom_20_percent_points = sorted_points[:int(0.2 * len(sorted_points))]

    # average of the bottom 20%
    average_of_bottom_20_percent = bottom_20_percent_points.mean()

    # top 20% of the prices
    top_20_percent_points = sorted_points[int(0.8 * len(sorted_points)):]
    # average of the top 20%
    average_of_top_20_percent = top_20_percent_points.mean()

    recent_lower_upper[unique] = (average_of_bottom_20_percent, average_of_top_20_percent)

294 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Then from here, you might calculate range or a ratio to determine variance for a given area and then find standouts:

In [53]:
for unique in uniques:

    points = df[df[which_unique_col] == unique]['total_points']
    sorted_points = points.sort_values()

    # bottom 20% of the prices
    bottom_20_percent_points = sorted_points[:int(0.2 * len(sorted_points))]

    # average of the bottom 20%
    average_of_bottom_20_percent = bottom_20_percent_points.mean()

    # top 20% of the prices
    top_20_percent_points = sorted_points[int(0.8 * len(sorted_points)):]
    # average of the top 20%
    average_of_top_20_percent = top_20_percent_points.mean()

    recent_lower_upper[unique] = (average_of_bottom_20_percent, average_of_top_20_percent)
    
recent_lower_upper
ratios = {}
ranges = {}

In [54]:
%%timeit
for color in recent_lower_upper:
    total_range = recent_lower_upper[color][1] - recent_lower_upper[color][0]
    ratio = recent_lower_upper[color][1] / recent_lower_upper[color][0]
    #confirm ratio is a number:
    if not np.isnan(ratio) and not np.isnan(total_range):
        #print(f"City: {city}, ratio: {ratio}, total_range: {total_range}")

        ratios[color] = ratio
        ranges[color] = total_range

1.25 ms ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
for color in recent_lower_upper:
    total_range = recent_lower_upper[color][1] - recent_lower_upper[color][0]
    ratio = recent_lower_upper[color][1] / recent_lower_upper[color][0]
    #confirm ratio is a number:
    if not np.isnan(ratio) and not np.isnan(total_range):
        #print(f"City: {city}, ratio: {ratio}, total_range: {total_range}")

        ratios[color] = ratio
        ranges[color] = total_range

# top 5 ratios:
sorted_ratios = sorted(ratios.items(), key=lambda x: x[1], reverse=True)
# top 5 ranges:
sorted_ranges = sorted(ranges.items(), key=lambda x: x[1], reverse=True)

print(f"Top 5 ratios: {sorted_ratios[:5]}")
print(f"Top 5 ranges: {sorted_ranges[:5]}")

Top 5 ratios: [('Satin Radiant Green', 21.857142857142858), ('Satin Light Black', 17.1), ('Matte Chromatic White', 17.01818181818182), ('Iridescent Racing Yellow', 14.850000000000001), ('Matte Dark Black', 14.303030303030303)]
Top 5 ranges: [('Gloss Light Red', 468.3), ('Matte Racing Black', 459.6136363636364), ('Satin Deep Blue', 454.125), ('Matte Radiant Black', 453.9818181818182), ('Iridescent Racing Yellow', 452.43333333333334)]


In [58]:
# find the earliest date of transfer:
df['total_points'].max()

525

In [59]:
%%timeit
df_sorted = df.sort_values(by=['total_points'], ascending=False)
# sort the dataframe by price:

1.75 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
df_sorted = df.sort_values(by=['total_points'], ascending=False)
df_sorted.tail()

,Unnamed: 0,year,team,best_finish,points_avg,total_points,color_type,car_color,good_res,prob_good_res
6185,6185,2003,leading,9,1,21,Matte Pale Blue,Iridescent,no,0.040337
3873,3873,2000,leading,13,1,21,Matte Dark Blue,Red,yes,0.426702
8567,8567,1992,leading,12,1,21,Iridescent Radiant Yellow,Semi-gloss,yes,0.798149
663,663,2007,leading,2,1,21,Matte Chromatic Green,Green,yes,0.480424
2371,2371,1990,midfield,17,1,21,Satin Dark White,Chromatic,yes,0.840277
